In [ ]:
import numpy as np
import gym

In [ ]:
env = gym.make('FrozenLake8x8-v0')

In [ ]:
#function to sample action according to soft policy
def sample_action(policy):
    return np.random.choice(len(policy),p=policy)

In [ ]:
def mc_on_policy(n_episodes,env,gamma,epsilon):
    
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    #initialize action-value function
    Q = np.zeros((n_states,n_actions))
    
    #initialize state-action visit counter
    N = np.zeros((n_states,n_actions))
    
    #initialize E-soft policy
    pi = np.ones((n_states,n_actions))/4
    
    for episode in range(n_episodes):
        observation = env.reset()
        state_action_reward_tuples = []
        t = 0
        done = False
        while not done:
            action = sample_action(pi[observation])
            old_observation = observation
            observation, reward, done, info = env.step(action)
            state_action_reward_tuples.append((observation,action,reward))
            t = t+1
            if done or t == 99:
                break
        G = 0
        visited_states = []
        for state,action,reward in state_action_reward_tuples[::-1]:
            visited_states.append(state)
            G = reward + gamma * G
            N[state][action] += 1
            Q[state][action] = (Q[state][action] +G) / N[state][action]

    for state in visited_states:
        best_value = -10000000000
        best_action = None
        for action in range(n_actions):
            if Q[state][action]>best_value:
                best_value = Q[state][action]
                best_action = action
        
        for action in range(n_actions):
            pi[state,action] = epsilon/4
            pi[state,best_action] = 1 - epsilon + epsilon/4
            
    return pi,Q

In [ ]:
gamma = 0.9
epsilon = 0.8
pi, Q = mc_on_policy(20000,env,gamma,epsilon)

In [ ]:
pi

In [ ]:
Q